In [1]:
import tensorflow as tf
from keras import backend as K
import os
import pandas as pd
import cv2
import numpy as np
import sys
sys.path.insert(0, "../")
from deeparuco.impl.utils import IoU, match_rois
from deeparuco.impl.losses import weighted_loss
from tensorflow.keras.models import load_model
from ultralytics import YOLO
import matplotlib.pyplot as plt
import csv
import time
import json
from shapely.geometry import Polygon
from deeparuco.impl.utils import ordered_corners, custom_marker_from_corners
from deeparuco.impl.customTag import find_id
from tensorflow.keras.saving import register_keras_serializable
from keras.metrics import mean_absolute_error

def calculate_iou(boxA, boxB):    
    polyA = Polygon(boxA)
    polyB = Polygon(boxB)
    
    if not polyA.is_valid or not polyB.is_valid:
        return 0.0
    
    intersection_area = polyA.intersection(polyB).area
    union_area = polyA.union(polyB).area
    
    return intersection_area / union_area if union_area > 0 else 0.0

def is_bbox_in_bounds(bbox_corners, h, w):
    marker_w = bbox_corners[2][0]-bbox_corners[0][0]
    marker_h = bbox_corners[2][1]-bbox_corners[0][1]
    for x, y in bbox_corners:
        if not (-marker_w/14 <= x <= w+marker_w/14 and -marker_h/14 <= y <= h+marker_h/14):
            return False
    return True  

@register_keras_serializable(package="Custom", name="extract_border")
def extract_border(x):
    # Extracting the outer border elements in clockwise order, starting from [0, 0] and ending at [1, 0]
    top_row = x[:, 0, :]         # First row (top border)
    bottom_row = x[:, -1, :]     # Last row (bottom border)
    left_column = x[:, 1:-1, 0]  # Left column, excluding top and bottom corners
    right_column = x[:, 1:-1, -1]  # Right column, excluding top and bottom corners
    
    # Concatenate the border elements in a specific clockwise order
    outer_border = tf.concat([
        top_row,                   # From [0, 0] to [0, 13]
        right_column,              # From [1, 13] to [12, 13]
        bottom_row[:, ::-1],       # From [13, 13] to [13, 0] (reverse order)
        left_column[::-1]          # From [12, 0] to [1, 0] (reverse order)
    ], axis=1)

    return outer_border

norm = lambda x: (x - np.min(x)) / (np.max(x) - np.min(x) + 1e-9)


2025-04-10 10:53:02.430697: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-10 10:53:02.544862: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-10 10:53:02.581359: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-10 10:53:02.811633: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-10 10:53:04.399583: W tensorflow/compiler/tf2

WARNING ⚠️ Python>=3.10 is required, but Python==3.9.5 is currently installed 


In [15]:
detector = YOLO('../models/nested2_detector_n/best.pt')
regressor = load_model("../models/nested2_regressor/corner_nested.keras",
    custom_objects={"weighted_loss": weighted_loss},)
decoder = load_model("../models/nested2_decoder/decoder.keras")
# decoder = load_model("/home/su/deeparuco/models_mine/reflection/decoder/decoder_new0.keras")

@tf.function(reduce_retracing=True)
def refine_corners(crops):
    return regressor(crops)

@tf.function(reduce_retracing=True)
def decode_markers(markers):
    return decoder(markers)

One image

In [16]:
import numpy as np
import cv2
from random import random, randint

def id_to_bits(id):
    txt_file = '/home/su/Infrared_apriltag_detection/deeparuco/codebook/apriltagCustom52h12_codebook_orig.txt'
    result = 0
    with open(txt_file, 'r') as infile:
        for line in infile:
            row = line.strip().split(',')
            if int(row[0]) == id:
                binary_string = ''.join(map(str, row[1:53]))
                integer = int(binary_string, 2)
                result = [float(bit) for bit in format(integer, '052b')]  
    return result


def rotate_ccw(border_points):
    """
    Rotate a 1D array of 52 border points counterclockwise by 90 degrees.
    """
    # Split the 1D array into the respective parts of the border (for 14x14 grid):
    top_row = border_points[:14]               # First 14 elements (top row)
    right_column = border_points[14:26]        # Next 12 elements (right column)
    bottom_row = border_points[26:40]          # Next 14 elements (bottom row, reversed)
    left_column = border_points[40:52]         # Last 12 elements (left column, reversed)

    # Rotate the points: top -> left, left -> bottom, bottom -> right, right -> top
    new_top_row = right_column                 # Right column becomes the new top row
    new_right_column = bottom_row[::-1]        # Bottom row becomes the new right column
    new_bottom_row = left_column               # Left column becomes the new bottom row
    new_left_column = top_row[::-1]            # Top row becomes the new left column

    # Concatenate the rotated border points back into a 1D array
    rotated_border_points = np.concatenate([new_top_row, new_right_column, new_bottom_row, new_left_column])

    return rotated_border_points


ids_as_bits = [id_to_bits(i) for i in range(250)]
def find_id_new(bits):

    rot0   = bits.flatten()
    rot90  = rotate_ccw(rot0)
    rot180 = rotate_ccw(rot90)
    rot270 = rotate_ccw(rot180)
    rot0_str = ''.join(map(str, map(int, rot0)))
    rot90_str = ''.join(map(str, map(int, rot90)))
    rot180_str = ''.join(map(str, map(int, rot180)))
    rot270_str = ''.join(map(str, map(int, rot270)))
    print(rot0_str)
    print(rot90_str)
    print(rot180_str)
    print(rot270_str)
    print(ids_as_bits)
    print(np.sum(np.abs(rot0 - ids_as_bits[0])))

    distances = [int(np.min([np.sum(np.abs(rot0 - check_bits)),
                np.sum(np.abs(rot90 - check_bits)),
                np.sum(np.abs(rot180 - check_bits)),
                np.sum(np.abs(rot270 - check_bits))])) 
                for check_bits in ids_as_bits]
    
    id = int(np.argmin(distances))

    return (id, distances[id])


In [22]:
im_path = '../dataset/Apr1_2025/img_8bit/image_135.jpg'
im = cv2.imread(im_path)
h, w = im.shape[:2]
im_size = h * w
    
detections = detector(im, verbose=False, iou=0.5, conf=0.03)[0].cpu().boxes

xyxy = [
    [
        int(max(det[0] - (0.2 * (det[2] - det[0]) + 0.5), 0)),
        int(max(det[1] - (0.2 * (det[3] - det[1]) + 0.5), 0)),
        int(min(det[2] + (0.2 * (det[2] - det[0]) + 0.5), im.shape[1] - 1)),
        int(min(det[3] + (0.2 * (det[3] - det[1]) + 0.5), im.shape[0] - 1)),
    ]
    for det in [
        [int(val) for val in det.xyxy.cpu().numpy()[0]] for det in detections
    ]
]
crops_ori = [
        cv2.resize(im[det[1] : det[3], det[0] : det[2]], (128, 128)) for det in xyxy
    ]


for i in range(len(crops_ori)):
    cv2.imwrite(f"crop_{i}.png", crops_ori[i])

crops = [norm(crop) for crop in crops_ori]
crop = crops[0]
crops_norm = [crop]

# crops = crops_ori.copy()
# corners = refine_corners(np.array(crops)).numpy()
# corners = [[(pred[i], pred[i + 1]) for i in range(0, 8, 2)] for pred in corners]
norm_corners = refine_corners(np.array(crops_norm)).numpy()
corners = [[(pred[i], pred[i + 1]) for i in range(0, 8, 2)] for pred in norm_corners]

# Ensure corners are ordered
corners = [
    ordered_corners([c[0] for c in cs], [c[1] for c in cs]) for cs in corners
]

markers = []

for crop, cs in zip(crops_ori, corners):
    marker = custom_marker_from_corners(crop, cs, 128)

    # Grayscale and normalize

    markers.append(norm(cv2.cvtColor(marker, cv2.COLOR_BGR2GRAY)))

for i in range(len(markers)):
    cv2.imwrite(f"marker_{i}.png", markers[i] * 255.0)

# Get ids from markers

decoder_out = np.round(decode_markers(np.array(markers)).numpy())
ids, dists = zip(*[find_id_new(out) for out in decoder_out])

1100100101011101101010100011101101010011111111011011
0110101010001100101011011111111101101111101010010011
0010101101111111011011111110101001001111000101010110
1101101111111111001001010100010101011011111011010100
[[1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0], [1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0], [1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0], [

In [19]:
print(ids, dists) 

(1,) (4,)


In [ ]:
Middle 1100100101010111100011000111000111001001111111011011
true   1100100101010111100011000111000101001001111010111011


In [ ]:
eval_path = '../dataset/nested2/Apr1_eval_model_n.csv'
im_dir    = '../dataset/nested2/images/test/'
start_time = time.time()

with open(eval_path, 'a', newline='') as eval_file:
	csv_writer = csv.writer(eval_file)
	csv_writer.writerow(['image', 'id'])

	for im_name in os.listdir(im_dir):	# Read image and lable
		im_path = im_dir + im_name
		name = im_name.split('.')[0]

		im = cv2.imread(im_path)
		h, w = im.shape[:2]
		im_size = h * w
		IDs = []
		bboxes = []
		reflections = []
		sizes = []
		sizes_ratio = []
		locations = ['outer', 'middle', 'inner']
			
		# Predict bbox
		# results = detector(im)[0]
		detections = detector(im, verbose=False, iou=0.5, conf=0.03)[0].cpu().boxes

		xyxy = [
			[
				int(max(det[0] - (0.2 * (det[2] - det[0]) + 0.5), 0)),
				int(max(det[1] - (0.2 * (det[3] - det[1]) + 0.5), 0)),
				int(min(det[2] + (0.2 * (det[2] - det[0]) + 0.5), im.shape[1] - 1)),
				int(min(det[3] + (0.2 * (det[3] - det[1]) + 0.5), im.shape[0] - 1)),
			]
			for det in [
				[int(val) for val in det.xyxy.cpu().numpy()[0]] for det in detections
			]
		]
		crops_ori = [
				cv2.resize(im[det[1] : det[3], det[0] : det[2]], (64, 64)) for det in xyxy
			]
		
		# for i in range(len(crops_ori)):
		# 	cv2.imwrite(f"crop_{name}_{i}.png", crops_ori[i])
		
		crops = [norm(crop) for crop in crops_ori]
		crop = crops[0]
		crops_norm = [crop]

		norm_corners = refine_corners(np.array(crops_norm)).numpy()
		corners = [[(pred[i], pred[i + 1]) for i in range(0, 8, 2)] for pred in norm_corners]

		# Ensure corners are ordered
		corners = [
			ordered_corners([c[0] for c in cs], [c[1] for c in cs]) for cs in corners
		]

		markers = []

		for crop, cs in zip(crops_ori, corners):
			marker = custom_marker_from_corners(crop, cs, 128)

			# Grayscale and normalize
			markers.append(norm(cv2.cvtColor(marker, cv2.COLOR_BGR2GRAY)))

		# for i in range(len(markers)):
		# 	cv2.imwrite(f"marker_{i}.png", markers[i] * 255.0)
		# Get ids from markers

		decoder_out = np.round(decode_markers(np.array(markers)).numpy())
		ids, dists = zip(*[find_id(out) for out in decoder_out])
		for id in ids:
			csv_writer.writerow([name, id])		
			
				

end_time = time.time()